In [8]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Path to data folder
directory = '../GATSol/dataset/'

# Load CSVs
df_train = pd.read_pickle(directory + 'eSol_train.pkl')
df_test = pd.read_pickle(directory + 'eSol_test.pkl')

continuous_column = 'solubility'  # Replace with your target column name
binary_column = 'binary_solubility'

# Identify scalar features
scalar_features = [
    col for col in df_train.columns
    if pd.api.types.is_numeric_dtype(df_train[col]) and col != continuous_column and col != binary_column
]

# Extract scalar features and target for regression and classification
X_train = df_train[scalar_features]
y_train_continuous = df_train[continuous_column]
y_train_binary = df_train[binary_column]
X_test = df_test[scalar_features]
y_test_continuous = df_test[continuous_column]
y_test_binary = df_test[binary_column]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
print(X_train_scaled_df.shape)
X_train_scaled_df.head()


(2019, 67)


,molecular_weight,aromaticity,gravy,isoelectric_point,length,Hydrophobicity_ARGP820101-G1,Hydrophobicity_ARGP820101-G2,Hydrophobicity_ARGP820101-G3,Hydrophobicity_CASG920101-G1,Hydrophobicity_CASG920101-G2,...,mean_contacts_per_residue,solvent_exposed_fraction,N_atom_type_proportion,C_atom_type_proportion,O_atom_type_proportion,S_atom_type_proportion,polar_exposed_residue_proportion,nonpolar_exposed_residue_proportion,positive_exposed_residue_proportion,negative_exposed_residue_proportion
0,-1.319825,5.116634,4.855120,0.712178,-1.338477,-4.882840,-1.552149,6.088027,-5.171033,2.092709,...,-1.081553,-2.669630,-3.970462,6.264260,-4.504737,0.033367,-2.294576,3.959105,-1.893561,-2.649554
1,2.397372,0.468661,0.313400,1.588408,2.346673,-0.844988,0.240913,0.587177,-0.121754,0.173917,...,0.277000,0.396178,-0.101950,0.916193,-0.989096,-0.255787,-0.840569,0.449496,0.495280,-0.368379
2,-0.362025,0.998023,-0.295374,0.082699,-0.394581,-0.497344,-0.433464,0.872151,-0.094455,-0.454403,...,0.376910,0.350622,0.746207,0.256970,-1.166814,0.999747,-0.757406,0.514899,0.484577,-0.457816
3,0.656888,-0.098094,0.154044,-0.652186,0.747137,0.560535,1.190676,-1.625646,-0.487905,1.264258,...,1.112271,0.747016,0.838893,-0.870924,0.436635,-0.195440,0.108377,0.757118,-0.497346,-0.677186
4,0.942892,0.237995,-0.116025,-0.692881,1.001480,0.782001,-0.047870,-0.703667,0.000016,-0.060356,...,0.680268,0.886290,0.018547,-0.493950,0.640156,-0.186341,0.916508,0.136768,-0.792905,-0.529838


In [10]:
# ---- Linear Regression ----
print("\n=== Linear Regression ===")
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train_continuous)

# Get coefficients and sort by importance
linear_coefficients = linear_model.coef_
linear_coef_df = pd.DataFrame({
    'Feature': scalar_features,
    'Coefficient': linear_coefficients
}).sort_values(by='Coefficient', key=abs, ascending=False)

# Print sorted coefficients
print("\nSorted Coefficients (Linear Regression):")
print(linear_coef_df)


=== Linear Regression ===

Sorted Coefficients (Linear Regression):
                               Feature  Coefficient
45                           num_atoms     3.754350
46                          total_mass    -2.707929
0                     molecular_weight    -1.541898
61              O_atom_type_proportion     0.725713
59              N_atom_type_proportion    -0.611208
..                                 ...          ...
15        Hydrophobicity_FASG890101-G2    -0.004357
55                       b_factors_min     0.003076
50                 bounding_box_volume     0.001675
26  Normalized van der Waals Volume-G1    -0.000721
48                          num_chains     0.000000

[67 rows x 2 columns]


In [11]:
# Evaluate the model
linear_r_squared = linear_model.score(X_test_scaled, y_test_continuous)
print(f"\nR^2 Score on Test Set: {linear_r_squared:.4f}")



R^2 Score on Test Set: 0.4291


In [12]:
df_train.columns

Index(['gene', 'solubility', 'sequence', 'embedding', 'binary_solubility',
       'molecular_weight', 'aromaticity', 'gravy', 'isoelectric_point',
       'length', 'aac', 'Hydrophobicity_ARGP820101-G1',
       'Hydrophobicity_ARGP820101-G2', 'Hydrophobicity_ARGP820101-G3',
       'Hydrophobicity_CASG920101-G1', 'Hydrophobicity_CASG920101-G2',
       'Hydrophobicity_CASG920101-G3', 'Hydrophobicity_ENGD860101-G1',
       'Hydrophobicity_ENGD860101-G2', 'Hydrophobicity_ENGD860101-G3',
       'Hydrophobicity_FASG890101-G1', 'Hydrophobicity_FASG890101-G2',
       'Hydrophobicity_FASG890101-G3', 'Hydrophobicity_PONP930101-G1',
       'Hydrophobicity_PONP930101-G2', 'Hydrophobicity_PONP930101-G3',
       'Hydrophobicity_PRAM900101-G1', 'Hydrophobicity_PRAM900101-G2',
       'Hydrophobicity_PRAM900101-G3', 'Hydrophobicity_ZIMJ680101-G1',
       'Hydrophobicity_ZIMJ680101-G2', 'Hydrophobicity_ZIMJ680101-G3',
       'Normalized van der Waals Volume-G1',
       'Normalized van der Waals Volume-G2

In [13]:
test_embeddings = df_test[["gene", "embedding", "blosum62_embedding"]]
train_embeddings = df_train[["gene", "embedding", "blosum62_embedding"]]
train_embeddings

,gene,embedding,blosum62_embedding
0,aaeX,"[[0.025740903, -0.06068451, 0.04562243, -0.098...","[5, 4, 4, 6, 7, 4, 4, 4, 4, 6, 6, 4, 4, 6, 7, ..."
1,aas,"[[-0.037975986, -0.046647176, -0.014576814, 0....","[5, 4, 6, 4, 6, 6, 5, 6, 4, 9, 5, 4, 4, 7, 5, ..."
2,aat,"[[0.037574537, -0.042643685, 0.05279441, -0.02...","[5, 5, 4, 4, 5, 4, 4, 5, 8, 4, 4, 4, 6, 7, 4, ..."
3,abgA,"[[-0.00765643, -0.13189432, 0.014674729, -0.01...","[5, 5, 4, 4, 6, 5, 6, 4, 6, 4, 4, 4, 7, 5, 4, ..."
4,abgB,"[[0.0036755833, -0.15647556, -0.01813248, 0.04...","[5, 5, 5, 4, 7, 5, 6, 4, 6, 6, 4, 4, 5, 4, 6, ..."
...,...,...,...
2014,zapA,"[[-0.017177405, -0.01794975, 0.029380163, -0.0...","[5, 4, 4, 5, 7, 4, 6, 4, 5, 4, 6, 6, 5, 4, 4, ..."
2015,zntA,"[[-0.021385752, -0.07058593, 0.024256472, 0.03...","[5, 4, 5, 7, 6, 6, 8, 6, 5, 5, 4, 7, 5, 6, 4, ..."
2016,znuA,"[[-0.0033216071, -0.061825093, 0.004684513, 0....","[5, 4, 8, 5, 5, 5, 4, 4, 6, 4, 4, 4, 4, 4, 4, ..."
2017,zur,"[[0.035278082, -0.063037135, -0.0353482, 0.028...","[5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 4, 5, 5, ..."


In [29]:
import pandas as pd
import numpy as np

def process_file_with_blosum_fixed(input_file, output_file, blosum_matrix, embedding_size=50):
    """
    Adds fixed-length BLOSUM embeddings to a DataFrame from a CSV or pickle file.

    Parameters:
        input_file (str): Path to the input CSV or pickle file.
        output_file (str): Path to save the output file.
        blosum_matrix (dict): BLOSUM matrix to calculate embeddings.
        embedding_size (int): Fixed size for the embeddings (default = 50).

    Returns:
        pd.DataFrame: DataFrame with fixed-length BLOSUM embeddings added.
    """
    # Load the input file
    if input_file.endswith('.csv'):
        df = pd.read_csv(input_file)
    elif input_file.endswith('.pkl'):
        df = pd.read_pickle(input_file)
    else:
        raise ValueError("Unsupported file format. Only .csv and .pkl are supported.")

    # Ensure the file has a 'sequence' column
    if 'sequence' not in df.columns:
        raise ValueError("The input file must contain a 'sequence' column.")

    # Define a function to calculate fixed-length BLOSUM embeddings
    def calculate_blosum62_embedding(sequence):
        embedding = [blosum_matrix.get((aa, aa), 0) for aa in sequence]  # Diagonal BLOSUM scores
        if len(embedding) < embedding_size:
            # Zero-pad to fixed size
            embedding += [0] * (embedding_size - len(embedding))
        elif len(embedding) > embedding_size:
            # Truncate to fixed size
            embedding = embedding[:embedding_size]
        return embedding

    # Add fixed-length BLOSUM embedding column
    df['blosum62_embedding'] = df['sequence'].apply(calculate_blosum62_embedding)

    # Save the updated DataFrame
    if output_file.endswith('.csv'):
        df.to_csv(output_file, index=False)
    elif output_file.endswith('.pkl'):
        df.to_pickle(output_file)
    else:
        raise ValueError("Unsupported output file format. Only .csv and .pkl are supported.")

    return df

In [37]:
# Example Usage
if __name__ == "__main__":
    # Define the BLOSUM62 matrix
    blosum62 = {
        ('A', 'A'): 4, ('R', 'R'): 5, ('N', 'N'): 6, ('D', 'D'): 6, ('C', 'C'): 9,
        ('Q', 'Q'): 5, ('E', 'E'): 5, ('G', 'G'): 6, ('H', 'H'): 8, ('I', 'I'): 4,
        ('L', 'L'): 4, ('K', 'K'): 5, ('M', 'M'): 5, ('F', 'F'): 6, ('P', 'P'): 7,
        ('S', 'S'): 4, ('T', 'T'): 5, ('W', 'W'): 11, ('Y', 'Y'): 7, ('V', 'V'): 4
    }

    input_path = "../GATSol/dataset/eSol_train_blosum.pkl"  # Replace with your input file path
    output_path = "../GATSol/dataset/eSol_train_blosum.pkl"  # Replace with your output file path

    updated_df = process_file_with_blosum_fixed(input_path, output_path, blosum62, embedding_size=50)
    print("Updated file saved to:", output_path)

Updated file saved to: ../GATSol/dataset/eSol_train_blosum.pkl


In [38]:
import pandas as pd

# Load the updated DataFrame
output_path = "../GATSol/dataset/eSol_train_blosum.pkl"
updated_df = pd.read_pickle(output_path)

# Inspect the first few rows
updated_df.head()


,gene,solubility,sequence,embedding,binary_solubility,molecular_weight,aromaticity,gravy,isoelectric_point,length,...,mean_contacts_per_residue,solvent_exposed_fraction,N_atom_type_proportion,C_atom_type_proportion,O_atom_type_proportion,S_atom_type_proportion,polar_exposed_residue_proportion,nonpolar_exposed_residue_proportion,positive_exposed_residue_proportion,negative_exposed_residue_proportion
0,aaeX,0.34,MSLFPVIVVFGLSFPPIFFELLLSLAIFWLVRRVLVPTGIYDFVWH...,"[[0.025740903, -0.06068451, 0.04562243, -0.098...",0,7846.4890,0.223881,1.485075,7.824157,67.0,...,3.358209,0.872340,0.145161,0.704301,0.145161,0.005376,0.101695,0.762712,0.067797,0.033898
1,aas,0.07,MLFSFFRNLCRVLYRVRVTGDTQALKGERVLITPNHVSFIDGILLG...,"[[-0.037975986, -0.046647176, -0.014576814, 0....",0,80699.0443,0.091794,-0.042281,9.312345,719.0,...,3.534075,0.982405,0.174112,0.643862,0.177278,0.004749,0.161744,0.554149,0.147679,0.105485
2,aat,0.08,MRLVQLSRHSIAFPSPEGALREPNGLLALGGDLSPARLLMAYQRGI...,"[[0.037574537, -0.042643685, 0.05279441, -0.02...",0,26618.2104,0.106838,-0.247009,6.755053,234.0,...,3.547009,0.980769,0.180459,0.636412,0.175654,0.007475,0.165179,0.558036,0.147321,0.102679
3,abgA,0.31,MESLNQFVNSLAPKLSHWRRDFHHYAESGWVEFRTATLVAEELHQL...,"[[-0.00765643, -0.13189432, 0.014674729, -0.01...",0,46587.6743,0.075688,-0.095872,5.506925,436.0,...,3.642202,0.995000,0.181153,0.623666,0.190302,0.004880,0.200935,0.572430,0.114486,0.095794
4,abgB,0.49,MQEIYRFIDDAIEADRQRYTDIADQIWDHPETRFEEFWSAEHLASA...,"[[0.0036755833, -0.15647556, -0.01813248, 0.04...",0,52193.0071,0.085239,-0.186694,5.437809,481.0,...,3.586279,1.000000,0.175014,0.627926,0.192161,0.004899,0.234310,0.535565,0.104603,0.100418


In [39]:
# Check the embedding dimensions
updated_df['blosum62_embedding'] = updated_df['blosum62_embedding'].apply(lambda x: len(x))
print("Unique embedding dimensions:", updated_df['blosum62_embedding'].unique())


Unique embedding dimensions: [50]


In [40]:
# Print the first few embeddings
print(updated_df['blosum62_embedding'].iloc[0])  # First row
print(updated_df['blosum62_embedding'].iloc[1])  # Second row


50
50


In [ ]:
import numpy as np

# Flatten ESM embeddings
train_embeddings['flattened_embedding'] = train_embeddings['embedding'].apply(
    lambda x: np.array(x).flatten()
)
test_embeddings['flattened_embedding'] = test_embeddings['embedding'].apply(
    lambda x: np.array(x).flatten()
)

# Validate dimensions
print("Train ESM dimensions:", train_embeddings['flattened_embedding'].apply(len).unique())
print("Test ESM dimensions:", test_embeddings['flattened_embedding'].apply(len).unique())


In [17]:
import numpy as np

# Flatten ESM embeddings
train_embeddings['flattened_embedding'] = train_embeddings['embedding'].apply(
    lambda x: np.array(x).flatten()
)
test_embeddings['flattened_embedding'] = test_embeddings['embedding'].apply(
    lambda x: np.array(x).flatten()
)

# Validate dimensions
print("Train ESM dimensions:", train_embeddings['flattened_embedding'].apply(len).unique())
print("Test ESM dimensions:", test_embeddings['flattened_embedding'].apply(len).unique())


Train ESM dimensions: [1280]
Test ESM dimensions: [1280]


In [18]:
# Convert flattened embeddings to NumPy arrays
X_train_embeddings = np.array(train_embeddings['flattened_embedding'].tolist())
X_test_embeddings = np.array(test_embeddings['flattened_embedding'].tolist())

# Check shapes
print("Train embeddings shape:", X_train_embeddings.shape)  # Should be (n_samples, 1280)
print("Test embeddings shape:", X_test_embeddings.shape)    # Should be (n_samples, 1280)


Train embeddings shape: (2019, 1280)
Test embeddings shape: (660, 1280)


In [21]:
# Combine scalar features and embeddings
X_train_combined = np.hstack([X_train_scaled, X_train_embeddings])
X_test_combined = np.hstack([X_test_scaled, X_test_embeddings])

# Validate shapes
print("Combined train shape:", X_train_combined.shape)
print("Combined test shape:", X_test_combined.shape)


Combined train shape: (2019, 1347)
Combined test shape: (660, 1347)


In [ ]:
# Train the XGBoost regressor
xgb_model = XGBRegressor(
    n_estimators=100,       # Number of trees
    learning_rate=0.1,      # Learning rate
    max_depth=5,            # Maximum depth
    random_state=42         # For reproducibility
)
xgb_model.fit(X_train_combined, y_train_continuous)

# Predict on the test set
y_pred = xgb_model.predict(X_test_combined)

# Evaluate the model
mse = mean_squared_error(y_test_continuous, y_pred)
r2 = r2_score(y_test_continuous, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")


Mean Squared Error: 0.0526
R^2 Score: 0.4914


In [23]:
# Extract feature importance
feature_importance = xgb_model.feature_importances_

# Create feature names (scalar features + embedding dimensions)
feature_names = scalar_features + [f"ESM_{i}" for i in range(X_train_embeddings.shape[1])]

# Create a DataFrame for interpretability
importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)

# Display the top 10 most important features
print("\nTop 10 Features by Importance:")
print(importance_df.head(10))



Top 10 Features by Importance:
                                  Feature  Importance
66    negative_exposed_residue_proportion    0.035025
46                             total_mass    0.019019
7            Hydrophobicity_ARGP820101-G3    0.017117
45                              num_atoms    0.009862
474                               ESM_407    0.009846
875                               ESM_808    0.008583
61                 O_atom_type_proportion    0.008459
952                               ESM_885    0.008356
4                                  length    0.008339
1046                              ESM_979    0.007924


In [ ]:
import numpy as np
train_embeddings = 
# Extract the BLOSUM embeddings as a NumPy array
X_train_blosum = np.stack(train_embeddings['blosum62_embedding'])
X_test_blosum = np.stack(test_embeddings['blosum62_embedding'])

# Verify shapes
print("Train BLOSUM embeddings shape:", X_train_blosum.shape)  # (n_samples, 50)
print("Test BLOSUM embeddings shape:", X_test_blosum.shape)
